In [9]:

import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
import pyarrow.compute as pc
import gc
from decimal import Decimal  # Add this import statement
import pyarrow.dataset as ds
import shutil
import gc
import time
import sqlalchemy as sa
import pyodbc



In [27]:
start_time = time.time()  # Start time


# Function to flush the cache
def flush_cache():
    gc.collect()

flush_cache()

In [17]:
output_folder_path = input("Enter the output folder path: ")
folder_path = r'D:\RISHIN\13_ILC_resolution\input\PARQUET_FILES'



In [25]:
folder_path_gr = r'D:\RISHIN\13_ILC_TASK\input\PARQUET_FILES_GR'


In [22]:
speriod=int(input("Enter the simulation period: "))
samples=int(input("Enter the number of samples: "))

In [18]:
parquet_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.parquet')]


In [26]:
parquet_files_gr = [os.path.join(folder_path_gr, f) for f in os.listdir(folder_path_gr) if f.endswith('.parquet')]


In [15]:
def delete_folder_and_files(folder_path):
    
    if os.path.exists(folder_path):
        # Delete all files inside the folder
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        
        # Delete the folder itself
        os.rmdir(folder_path)
        print(f'Successfully deleted the folder: {folder_path}')
    else:
        print(f'The folder {folder_path} does not exist.')


In [19]:
# Check if there are any Parquet files in the folder
if parquet_files:
    # Read the first Parquet file in chunks
    parquet_file = pq.ParquetFile(parquet_files[0])
    for batch in parquet_file.iter_batches(batch_size=1000):
        # Convert the first batch to a PyArrow Table
        table = pa.Table.from_batches([batch])
        
        # Convert the PyArrow Table to a Pandas DataFrame
        df = table.to_pandas()
        
        # Extract the first value of LocationName and split it by '_'
        location_name = df['LocationName'].iloc[0]
        country = location_name.split('_')[0]
        
        
        # Define the main folder path
        main_folder_path = os.path.join(output_folder_path, f'ILC2024_EUWS_PLA_WI_EP_{country}_EUR_Losses')
        
        # Define subfolders
        subfolders = ['Admin_1','Admin_1_Lob','Cresta','Cresta_1','EP', 'PLT', 'STATS']
        nested_folders = ['Lob', 'Portfolio']
        innermost_folders = ['GR', 'GU']
        
        # Create the main folder and subfolders
        for subfolder in subfolders:
            subfolder_path = os.path.join(main_folder_path, subfolder)
            os.makedirs(subfolder_path, exist_ok=True)
            
            for nested_folder in nested_folders:
                nested_folder_path = os.path.join(subfolder_path, nested_folder)
                os.makedirs(nested_folder_path, exist_ok=True)
                
                for innermost_folder in innermost_folders:
                    innermost_folder_path = os.path.join(nested_folder_path, innermost_folder)
                    os.makedirs(innermost_folder_path, exist_ok=True)
        
        print(f"Folders created successfully at {main_folder_path}")
        break  # Process only the first batch
else:
    print("No Parquet files found in the specified folder.")

Folders created successfully at D:\RISHIN\TESTING\TEST_3\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses


In [21]:
processing_folder_path = os.path.join(main_folder_path, 'processing')
resolution_folder_path = os.path.join(processing_folder_path, 'Resolution Added')


In [28]:
resolution_folder_path_gr = os.path.join(processing_folder_path, 'Resolution Added_gr')


In [24]:
def connect_to_database(server, database):
    connection_string = f'mssql+pyodbc://{server}/{database}?driver=SQL+Server+Native+Client+11.0'
    engine = sa.create_engine(connection_string)
    connection = engine.connect()
    return connection

def read_parquet_file(file_path):
    table = pq.read_table(file_path)
    return table

def fetch_database_data(connection):
    address_query = 'SELECT ADDRESSID, ADMIN1GEOID AS Admin1Id, Admin1Name, zone1GEOID AS CrestaId, Zone1 AS CrestaName FROM Address'
    address_df = pd.read_sql(address_query, connection)
    address_table = pa.Table.from_pandas(address_df)
    return address_table

def join_dataframes(parquet_table, address_table):
    parquet_df = parquet_table.to_pandas()
    address_df = address_table.to_pandas()
    df = parquet_df.merge(address_df, left_on='LocationId', right_on='ADDRESSID', how='left')
    return pa.Table.from_pandas(df)

def save_joined_dataframe(joined_table, output_file):
    pq.write_table(joined_table, output_file)
    print(f"Saved joined file to {output_file}")

def process_parquet_files(folder_path, output_folder, server, database):
    os.makedirs(output_folder, exist_ok=True)
    connection = connect_to_database(server, database)
    address_table = fetch_database_data(connection)

    parquet_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.parquet')]
    for file in parquet_files:
        parquet_table = read_parquet_file(file)
        joined_table = join_dataframes(parquet_table, address_table)
        output_file = os.path.join(output_folder, os.path.basename(file))
        save_joined_dataframe(joined_table, output_file)
        del parquet_table
        del joined_table
        gc.collect()

    connection.close()


output_folder = r'D:\RISHIN\TESTING\TEST_2'
server = 'localhost'
database = 'IED2024_EUWS_PC_MIX_EDM230_ILC_LOB_UPDATE_20240905'

process_parquet_files(folder_path, resolution_folder_path, server, database)

Saved joined file to D:\RISHIN\TESTING\TEST_3\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses\processing\Resolution Added\PLT_0_100.parquet
Saved joined file to D:\RISHIN\TESTING\TEST_3\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses\processing\Resolution Added\PLT_0_200.parquet
Saved joined file to D:\RISHIN\TESTING\TEST_3\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses\processing\Resolution Added\PLT_0_300.parquet
Saved joined file to D:\RISHIN\TESTING\TEST_3\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses\processing\Resolution Added\PLT_10_100.parquet
Saved joined file to D:\RISHIN\TESTING\TEST_3\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses\processing\Resolution Added\PLT_10_200.parquet
Saved joined file to D:\RISHIN\TESTING\TEST_3\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses\processing\Resolution Added\PLT_10_300.parquet
Saved joined file to D:\RISHIN\TESTING\TEST_3\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses\processing\Resolution Added\PLT_11_100.parquet
Saved joined file to D:\RISHIN\TESTING\TEST_3\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses\processing

In [29]:
process_parquet_files(folder_path_gr, resolution_folder_path_gr, server, database)

Saved joined file to D:\RISHIN\TESTING\TEST_3\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses\processing\Resolution Added_gr\PLT_0_100.parquet
Saved joined file to D:\RISHIN\TESTING\TEST_3\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses\processing\Resolution Added_gr\PLT_0_200.parquet
Saved joined file to D:\RISHIN\TESTING\TEST_3\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses\processing\Resolution Added_gr\PLT_0_300.parquet
Saved joined file to D:\RISHIN\TESTING\TEST_3\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses\processing\Resolution Added_gr\PLT_10_100.parquet
Saved joined file to D:\RISHIN\TESTING\TEST_3\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses\processing\Resolution Added_gr\PLT_10_200.parquet
Saved joined file to D:\RISHIN\TESTING\TEST_3\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses\processing\Resolution Added_gr\PLT_10_300.parquet
Saved joined file to D:\RISHIN\TESTING\TEST_3\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses\processing\Resolution Added_gr\PLT_11_100.parquet
Saved joined file to D:\RISHIN\TESTING\TEST_3\ILC2024_EUWS_PLA_WI_EP_BE_

In [ ]:
#Admin_1 Lob

In [ ]:

def process_parquet_files(parquet_files, export_path, filter_string, lob_id, speriod, samples, rps_values,parquet_file_path):
    partial_folder_path = os.path.join(processing_folder_path, 'partial')
    concatenated_folder_path = os.path.join(processing_folder_path, 'concatenated')
    os.makedirs(partial_folder_path, exist_ok=True)
    os.makedirs(concatenated_folder_path, exist_ok=True)

    # Initialize an empty list to store the results
    final_grouped_table_1 = []

    # Process each Parquet file individually
    for file in parquet_files:
        # Read the Parquet file into a PyArrow Table
        table = pq.read_table(file)
        
        # Filter the table based on the filter_string
        table = table.filter(pc.equal(table['LobName'], filter_string))
        # Skip if the filtered table is empty
        if len(table) == 0:
            continue

        grouped_table_1 = table.group_by(['EventId', 'PeriodId', 'EventDate','Admin1Name','Admin1Id']).aggregate([('Loss', 'sum')])
        grouped_table_1 = grouped_table_1.rename_columns(['EventId', 'PeriodId', 'EventDate','Admin1Name','Admin1Id','Sum_Loss'])
    
        # Write intermediate results to disk
        pq.write_table(grouped_table_1, os.path.join(partial_folder_path, f'grouped_table_1_{os.path.basename(file)}'))

    # Read all intermediate files and concatenate them
    intermediate_files_1 = [os.path.join(partial_folder_path, f) for f in os.listdir(partial_folder_path) if f.startswith('grouped_table_1_')]

    final_grouped_table_1 = [pq.read_table(f) for f in intermediate_files_1]

    final_table_1 = pa.concat_tables(final_grouped_table_1)

    # Perform final grouping and sorting
    f_grouped_table_1 = final_table_1.group_by(['EventId', 'PeriodId', 'EventDate','Admin1Name','Admin1Id']).aggregate([('Sum_Loss', 'sum')])
    sorted_final_table_1 = f_grouped_table_1.sort_by([('Sum_Loss_sum', 'descending')])
    pq.write_table(sorted_final_table_1, os.path.join(concatenated_folder_path, 'final_grouped_table_1.parquet'))

    # Delete all non-concatenated files
    for f in intermediate_files_1:
        os.remove(f)
    
    dataframe_1 = sorted_final_table_1.to_pandas()

    dataframe_2 = dataframe_1.groupby(['PeriodId'], as_index=False).agg({'Sum_Loss_sum': 'max'})
    dataframe_2.rename(columns={'Sum_Loss_sum': 'Max_Loss'}, inplace=True)
    dataframe_2 = dataframe_2.sort_values(by='Max_Loss', ascending=False).reset_index(drop=True)

    dataframe_2['rate'] = (1 / (speriod * samples))
    dataframe_2['cumrate'] = dataframe_2['rate'].cumsum().round(6)
    dataframe_2['RPs'] = (1 / dataframe_2['cumrate'])
    dataframe_2['TCE_OEP_1'] = ((dataframe_2['Max_Loss'] - dataframe_2['Max_Loss'].shift(-1)) * (dataframe_2['cumrate'] + dataframe_2['cumrate'].shift(-1)) * 0.5)
    dataframe_2['TCE_OEP_2'] = (dataframe_2['TCE_OEP_1'].shift().cumsum() * dataframe_2['RPs'])
    dataframe_2['TCE_OEP_Final'] = (dataframe_2['TCE_OEP_2'] + dataframe_2['Max_Loss'])

    dataframe_3 = dataframe_1.groupby(['PeriodId'], as_index=False).agg({'Sum_Loss_sum': 'sum'})
    dataframe_3.rename(columns={'Sum_Loss_sum': 'S_Sum_Loss'}, inplace=True)
    dataframe_3 = dataframe_3.sort_values(by='S_Sum_Loss', ascending=False).reset_index(drop=True)

    dataframe_3['rate'] = (1 / (speriod * samples))
    dataframe_3['cumrate'] = dataframe_3['rate'].cumsum().round(6)
    dataframe_3['RPs'] = (1 / dataframe_3['cumrate'])
    dataframe_3['TCE_AEP_1'] = ((dataframe_3['S_Sum_Loss'] - dataframe_3['S_Sum_Loss'].shift(-1)) * (dataframe_3['cumrate'] + dataframe_3['cumrate'].shift(-1)) * 0.5)
    dataframe_3['TCE_AEP_2'] = (dataframe_3['TCE_AEP_1'].shift().cumsum() * dataframe_3['RPs'])
    dataframe_3['TCE_AEP_Final'] = (dataframe_3['TCE_AEP_2'] + dataframe_3['S_Sum_Loss'])

    fdataframe_2 = pd.DataFrame()
    fdataframe_3 = pd.DataFrame()

    for value in rps_values:
        closest_index_2 = (dataframe_2['RPs'] - value).abs().idxmin()
        fdataframe_2 = pd.concat([fdataframe_2, dataframe_2.loc[[closest_index_2]]])
        fdataframe_3 = pd.concat([fdataframe_3, dataframe_3.loc[[closest_index_2]]])

    fdataframe_2.rename(columns={'Max_Loss': 'OEP', 'TCE_OEP_Final': 'TCE-OEP'}, inplace=True)
    columns_to_keep_2 = ['RPs']
    columns_to_melt_2 = ['OEP', 'TCE-OEP']
    melted_df_2 = fdataframe_2.melt(id_vars=columns_to_keep_2, value_vars=columns_to_melt_2, var_name='EPType', value_name='Loss')
    melted_df_2.rename(columns={'RPs': 'ReturnPeriod'}, inplace=True)
    final_df_2 = melted_df_2[['EPType', 'Loss', 'ReturnPeriod']]

    fdataframe_3.rename(columns={'S_Sum_Loss': 'AEP', 'TCE_AEP_Final': 'TCE-AEP'}, inplace=True)
    columns_to_keep_3 = ['RPs']
    columns_to_melt_3 = ['AEP', 'TCE-AEP']
    melted_df_3 = fdataframe_3.melt(id_vars=columns_to_keep_3, value_vars=columns_to_melt_3, var_name='EPType', value_name='Loss')
    melted_df_3.rename(columns={'RPs': 'ReturnPeriod'}, inplace=True)
    final_df_3 = melted_df_3[['EPType', 'Loss', 'ReturnPeriod']]

    final_df_EP_LOB_GU = pd.concat([final_df_2, final_df_3], ignore_index=True)
    new_ep_type_order = ["OEP", "AEP", "TCE-OEP", "TCE-AEP"]
    final_df_EP_LOB_GU['EPType'] = pd.Categorical(final_df_EP_LOB_GU['EPType'], categories=new_ep_type_order, ordered=True)
    final_df_EP_LOB_GU = final_df_EP_LOB_GU.sort_values(by=['EPType', 'ReturnPeriod'], ascending=[True, False]).reset_index(drop=True)

    # Add LobID and LobName columns
    final_df_EP_LOB_GU['LOBId'] = lob_id
    final_df_EP_LOB_GU['LOBName'] = filter_string
    final_df_EP_LOB_GU['LOBId'] = final_df_EP_LOB_GU['LobID'].apply(lambda x: Decimal(x))


    # Define the schema to match the required Parquet file schema
    schema = pa.schema([
        pa.field('EPType', pa.string(), nullable=True),
        pa.field('Loss', pa.float64(), nullable=True),
        pa.field('ReturnPeriod', pa.float64(), nullable=True),
        pa.field('Admin1Id',pa.decimal128(38, 0), nullable=True),
        pa.field('Admin1Name', pa.string(), nullable=True),
        pa.field('LOBId', pa.decimal128(38, 0), nullable=True),
        pa.field('LOBId', pa.string(), nullable=True),
    ])

    # Convert DataFrame to Arrow Table with the specified schema
    table = pa.Table.from_pandas(final_df_EP_LOB_GU, schema=schema)

    # Save to Parquet
    pq.write_table(table, parquet_file_path)

    print(f"Parquet file saved successfully at {parquet_file_path}")

In [ ]:


export_path =os.path.join(main_folder_path, 'EP', 'Lob','GU')
parquet_file_path_AUTO = os.path.join(export_path, 'ILC2024_EUWS_PLA_WI_EP_BE_EUR_EP_Lob_GU_1.parquet')
parquet_file_path_AGR = os.path.join(export_path, 'ILC2024_EUWS_PLA_WI_EP_BE_EUR_EP_Lob_GU_0.parquet')
parquet_file_path_COM = os.path.join(export_path, 'ILC2024_EUWS_PLA_WI_EP_BE_EUR_EP_Lob_GU_2.parquet')
parquet_file_path_IND = os.path.join(export_path, 'ILC2024_EUWS_PLA_WI_EP_BE_EUR_EP_Lob_GU_3.parquet')
parquet_file_path_SPER = os.path.join(export_path, 'ILC2024_EUWS_PLA_WI_EP_BE_EUR_EP_Lob_GU_4.parquet')
parquet_file_path_FRST= os.path.join(export_path, 'ILC2024_EUWS_PLA_WI_EP_BE_EUR_EP_Lob_GU_5.parquet')
parquet_file_path_GLH = os.path.join(export_path, 'ILC2024_EUWS_PLA_WI_EP_BE_EUR_EP_Lob_GU_6.parquet')

rps_values = [10000, 5000, 1000, 500, 250, 200, 100, 50, 25, 10, 5, 2]




try:
    process_parquet_files(parquet_files, export_path, 'AGR', 1, speriod, samples, rps_values, parquet_file_path_AGR)
except (NameError, AttributeError,ValueError) as e:
    print(f"Error processing AGR: {e}")
    pass

try:
    process_parquet_files(parquet_files, export_path, 'AUTO', 2, speriod, samples, rps_values, parquet_file_path_AUTO)
except (NameError, AttributeError,ValueError) as e:
    print(f"Error processing AUTO: {e}")
    pass

try:
    process_parquet_files(parquet_files, export_path, 'COM', 3, speriod, samples, rps_values, parquet_file_path_COM)
except (NameError, AttributeError,ValueError) as e:
    print(f"Error processing COM: {e}")
    pass

try:
    process_parquet_files(parquet_files, export_path, 'IND', 4, speriod, samples, rps_values, parquet_file_path_IND)
except (NameError, AttributeError,ValueError) as e:
    print(f"Error processing IND: {e}")
    pass

try:
    process_parquet_files(parquet_files, export_path, 'SPER', 5, speriod, samples, rps_values, parquet_file_path_SPER)
except (NameError, AttributeError,ValueError) as e:
    print(f"Error processing SPER: {e}")
    pass

try:
    process_parquet_files(parquet_files, export_path, 'FRST', 6, speriod, samples, rps_values, parquet_file_path_FRST)
except (NameError, AttributeError,ValueError) as e:
    print(f"Error processing FRST: {e}")
    pass

try:
    process_parquet_files(parquet_files, export_path, 'GLH', 7, speriod, samples, rps_values, parquet_file_path_GLH)
except (NameError, AttributeError,ValueError) as e:
    print(f"Error processing GLH: {e}")
    pass




# In[ ]:


#NEXT FOR GR


# In[77]:


export_path_gr =os.path.join(main_folder_path, 'EP', 'Lob','GR')
parquet_file_path_AUTO = os.path.join(export_path_gr, 'ILC2024_EUWS_PLA_WI_EP_BE_EUR_EP_Lob_GR_1.parquet')
parquet_file_path_AGR = os.path.join(export_path_gr, 'ILC2024_EUWS_PLA_WI_EP_BE_EUR_EP_Lob_GR_0.parquet')
parquet_file_path_COM = os.path.join(export_path_gr, 'ILC2024_EUWS_PLA_WI_EP_BE_EUR_EP_Lob_GR_2.parquet')
parquet_file_path_IND = os.path.join(export_path_gr, 'ILC2024_EUWS_PLA_WI_EP_BE_EUR_EP_Lob_GR_3.parquet')
parquet_file_path_SPER = os.path.join(export_path_gr, 'ILC2024_EUWS_PLA_WI_EP_BE_EUR_EP_Lob_GR_4.parquet')
parquet_file_path_FRST= os.path.join(export_path_gr, 'ILC2024_EUWS_PLA_WI_EP_BE_EUR_EP_Lob_GR_5.parquet')
parquet_file_path_GLH = os.path.join(export_path_gr, 'ILC2024_EUWS_PLA_WI_EP_BE_EUR_EP_Lob_GR_6.parquet')





try:
    process_parquet_files(parquet_files_gr, export_path, 'AGR', 1, speriod, samples, rps_values, parquet_file_path_AGR)
except (NameError, AttributeError,ValueError) as e:
    print(f"Error processing AGR: {e}")
    pass

try:
    process_parquet_files(parquet_files_gr, export_path, 'AUTO', 2, speriod, samples, rps_values, parquet_file_path_AUTO)
except (NameError, AttributeError,ValueError) as e:
    print(f"Error processing AUTO: {e}")
    pass

try:
    process_parquet_files(parquet_files_gr, export_path, 'COM', 3, speriod, samples, rps_values, parquet_file_path_COM)
except (NameError, AttributeError,ValueError) as e:
    print(f"Error processing COM: {e}")
    pass

try:
    process_parquet_files(parquet_files_gr, export_path, 'IND', 4, speriod, samples, rps_values, parquet_file_path_IND)
except (NameError, AttributeError,ValueError) as e:
    print(f"Error processing IND: {e}")
    pass

try:
    process_parquet_files(parquet_files_gr, export_path, 'SPER', 5, speriod, samples, rps_values, parquet_file_path_SPER)
except (NameError, AttributeError,ValueError) as e:
    print(f"Error processing SPER: {e}")
    pass

try:
    process_parquet_files(parquet_files_gr, export_path, 'FRST', 6, speriod, samples, rps_values, parquet_file_path_FRST)
except (NameError, AttributeError,ValueError) as e:
    print(f"Error processing FRST: {e}")
    pass

try:
    process_parquet_files(parquet_files_gr, export_path, 'GLH', 7, speriod, samples, rps_values, parquet_file_path_GLH)
except (NameError, AttributeError,ValueError) as e:
    print(f"Error processing GLH: {e}")
    pass

partial_folder_path = os.path.join(processing_folder_path, 'partial')
concatenated_folder_path = os.path.join(processing_folder_path, 'concatenated')

delete_folder_and_files(partial_folder_path)
delete_folder_and_files(concatenated_folder_path)
